# Evaluating scale sensitivity of stress

In [8]:
## custom
from utils import utils, vis
from utils import poly_point_isect as bo   ##bentley-ottmann sweep line
import criteria as C
import quality as Q
import gd2
import utils.weight_schedule as ws

## third party
import networkx as nx

from PIL import Image
from natsort import natsorted


## sys
from collections import defaultdict
import random
import time
from glob import glob
import math
import os
from pathlib import Path
import itertools
import pickle as pkl

## numeric
import numpy as np
import scipy.io as io
import torch
from torch import nn, optim
import torch.nn.functional as F


## vis
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from matplotlib.gridspec import GridSpec
from matplotlib.colors import LinearSegmentedColormap
from mpl_toolkits import mplot3d
from matplotlib import collections  as mc
from mpl_toolkits.mplot3d.art3d import Line3DCollection

## notebook
from IPython import display
from IPython.display import clear_output
# from tqdm.notebook import tqdm
from tqdm import tqdm

device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
device = 'cpu'
plt.style.use('ggplot')
# plt.style.use('seaborn-colorblind')

from utils.weight_schedule import SmoothSteps, Concat, SmoothStepSchedule


graphs = ['dodecahedron', 'tree_2_6', 'grid_12_24', 'spx_teaser', '494_bus', 'grid1', 'dwt_307', 'dwt_1005']

methods = [
    ('neato', 'neato'),
    ('sfdp', 'sfdp'),
    
    ('GD2 (ST)', 'stress'),
    ('GD2 (NP)', 'neighborhood_preservation'),
    
    ('GD2 (ST+IL)', 'stress+ideal_edge_length'),
    ('GD2 (ST+NP)', 'stress+neighborhood_preservation'),
    ('GD2 (ST+CR)', 'stress+crossings'),
    ('GD2 (ST+CAM)', 'stress+crossing_angle_maximization'),
    ('GD2 (ST+AR)', 'stress+aspect_ratio'),
    ('GD2 (ST+VR)',     'stress+vertex_resolution'),
    ('GD2 (ST+GB)',     'stress+gabriel'),
    
    ('GD2 (ST+IL+ANR)', 'stress+ideal_edge_length+angular_resolution'),
    ('GD2 (IL+NP+VR)',  'ideal_edge_length+neighborhood_preservation+vertex_resolution'),
    
]


## Compute layout quality, uncomment to recompute

In [3]:
import pandas as pd

import importlib
importlib.reload(C)
importlib.reload(Q)
importlib.reload(utils)
importlib.reload(vis)
import gd2
importlib.reload(gd2)
from gd2 import GD2

In [9]:
# %%time

criteria_all = [
    'stress',
    # 'ideal_edge_length',
    # 'neighborhood_preservation',
    # 'crossings',
    # 'crossing_angle_maximization',
    # 'aspect_ratio',
    # 'angular_resolution',
    # 'vertex_resolution',
    # 'gabriel',
]

quality_tables = {c:[
    [utils.criterion_to_title(c)],
    ['', *[g.replace('_', '-') for g in graphs]], 
    *[[ms[0]] for ms in methods]] 
    for c in criteria_all
}

for (i, (method_short, method)), (j,graph) in tqdm(list(itertools.product(enumerate(methods), enumerate(graphs)))):
    
    fn = f'./layouts/{graph}-{method}.pkl'
    with open(fn, 'rb') as f:
        result_dict = pkl.load(f)
    G = result_dict.get('G')
    pos_G = result_dict.get('pos_G')
    gd = GD2(G)
    pos = torch.tensor([pos_G[gd.i2k[i]] for i in range(len(G))])
    
    ## TODO give 'full advantange' to neato and sfdp
    ## on stress and ideal_edge_length
    if method in {'neato', 'sfdp'}:
        qualities = gd.evaluate(pos, qualities=criteria_all, mode='best_scale')
#         print(qualities['stress'], qualities['ideal_edge_length'])
    else:
        qualities = gd.evaluate(pos, qualities=criteria_all, mode='original')
    for c,q in qualities.items():
#         quality_tables[c][i+2].append(float(f'{q:.2f}'))
        quality_tables[c][i+2].append(q)


 83%|████████▎ | 86/104 [01:16<00:09,  1.88it/s]

In [16]:
# %%time

criteria_all = [
    'stress',
    # 'ideal_edge_length',
    # 'neighborhood_preservation',
    # 'crossings',
    # 'crossing_angle_maximization',
    # 'aspect_ratio',
    # 'angular_resolution',
    # 'vertex_resolution',
    # 'gabriel',
]

quality_tables_scaled = {c:[
    [utils.criterion_to_title(c)],
    ['', *[g.replace('_', '-') for g in graphs]], 
    *[[ms[0]] for ms in methods]] 
    for c in criteria_all
}

for (i, (method_short, method)), (j,graph) in tqdm(list(itertools.product(enumerate(methods), enumerate(graphs)))):
    
    fn = f'./layouts/{graph}-{method}.pkl'
    with open(fn, 'rb') as f:
        result_dict = pkl.load(f)
    G = result_dict.get('G')
    pos_G = result_dict.get('pos_G')
    gd = GD2(G)
    pos = torch.tensor([pos_G[gd.i2k[i]] for i in range(len(G))])
    
    ## TODO give 'full advantange' to neato and sfdp
    ## on stress and ideal_edge_length
    if method in {'neato', 'sfdp'}:
        qualities = gd.evaluate(pos, qualities=criteria_all, mode='best_scale')
#         print(qualities['stress'], qualities['ideal_edge_length'])
    else:
        qualities = gd.evaluate(pos, qualities=criteria_all, mode='best_scale')
    for c,q in qualities.items():
#         quality_tables[c][i+2].append(float(f'{q:.2f}'))
        quality_tables_scaled[c][i+2].append(q)

  1%|          | 1/104 [00:00<00:18,  5.60it/s]

neato dodecahedron
neato tree_2_6
neato grid_12_24


  3%|▎         | 3/104 [00:00<00:08, 11.76it/s]

neato spx_teaser
neato 494_bus


  5%|▍         | 5/104 [00:00<00:10,  9.57it/s]

...

 96%|█████████▌| 100/104 [01:31<00:02,  1.37it/s]

ideal_edge_length+neighborhood_preservation+vertex_resolution spx_teaser
ideal_edge_length+neighborhood_preservation+vertex_resolution 494_bus


 98%|█████████▊| 102/104 [01:31<00:01,  1.81it/s]

ideal_edge_length+neighborhood_preservation+vertex_resolution grid1
ideal_edge_length+neighborhood_preservation+vertex_resolution dwt_307
ideal_edge_length+neighborhood_preservation+vertex_resolution dwt_1005


100%|██████████| 104/104 [01:38<00:00,  1.05it/s]


In [18]:
mat1 = list()
for val in quality_tables['stress'][2:]:
    mat1.append(val[1:])

mat1 = np.array(mat1)
mat1

mat2 = list()
for val in quality_tables_scaled['stress'][2:]:
    mat2.append(val[1:])
mat2 = np.array(mat2)
for row in quality_tables_scaled['stress']:
    print(row)

['Stress']
['', 'dodecahedron', 'tree-2-6', 'grid-12-24', 'spx-teaser', '494-bus', 'grid1', 'dwt-307', 'dwt-1005']
['neato', 0.08134147731360619, 0.07783726549400836, 0.012811696842822708, 0.02713600532007563, 0.07607956020075818, 0.06176972744186493, 0.0832831469335954, 0.021279316333066168]
['sfdp', 0.0797211560875147, 0.13308516972864373, 0.02447798141388848, 0.0516847548405388, 0.0989621556655844, 0.07086976395761918, 0.08074983278954283, 0.029469892231659165]
['GD2 (ST)', 0.07949503833523776, 0.07781891674515576, 0.012828487718222932, 0.02681389206034175, 0.07124048947650795, 0.0617665569875385, 0.08246111425166115, 0.02221180758755616]
['GD2 (NP)', 0.15334976037218026, 0.12238675883668801, 0.06237162031952927, 0.18394656823919947, 0.1540913753674537, 0.1336825286734272, 0.15939180793553287, 0.1673372272208446]
['GD2 (ST+IL)', 0.08563848297380958, 0.08027643613707941, 0.013988727752287254, 0.03323974962472137, 0.07550453872190732, 0.06817887455680605, 0.0927623519047073, 0.0277128

In [26]:
from scipy.stats import rankdata

data = dict()

data['algorithms'] = [d[0] for d in quality_tables['stress'][2:]]
data['graphs'] = list()

for i,gname in zip(range(mat1.shape[1]), quality_tables['stress'][1][1:]):
    data['graphs'].append({
        "name": gname, 
        "ns-rank": [int(d) for d in rankdata(mat1[:,i])],
        "sns-rank": [int(d) for d in rankdata(mat2[:,i])]
    })
    print(gname)
    print("Ranking in paper", rankdata(mat1[:,i]))
    print("Ranking by SNS  ", rankdata(mat2[:,i]))
    print("-----------------")

dodecahedron
Ranking in paper [ 5.  3.  1. 12. 10. 11. 13.  9.  2.  7.  4.  8.  6.]
Ranking by SNS   [ 6.  3.  1. 11.  9. 12. 13. 10.  2.  8.  4.  5.  7.]
-----------------
tree-2-6
Ranking in paper [ 4. 12.  2. 13.  9. 10.  5.  1.  7.  6.  3.  8. 11.]
Ranking by SNS   [ 4. 13.  2. 12.  7. 10.  5.  1.  8.  6.  3.  9. 11.]
-----------------
grid-12-24
Ranking in paper [ 1.  9.  2. 13. 10.  6.  4.  7. 11.  5.  3.  8. 12.]
Ranking by SNS   [ 1. 10.  2. 13.  7.  6.  4.  9. 11.  5.  3.  8. 12.]
-----------------
spx-teaser
Ranking in paper [ 4. 10.  1. 13. 11.  9.  5.  6.  2.  7.  3.  8. 12.]
Ranking by SNS   [ 4. 11.  1. 13. 10.  9.  5.  6.  2.  7.  3.  8. 12.]
-----------------
494-bus
Ranking in paper [ 6. 10.  2. 13.  9. 12.  7.  4.  5.  3.  1.  8. 11.]
Ranking by SNS   [ 8. 11.  2. 13.  7. 10.  5.  4.  6.  3.  1.  9. 12.]
-----------------
grid1
Ranking in paper [ 2.  8.  1. 13. 12. 10.  9.  5.  6.  4.  3. 11.  7.]
Ranking by SNS   [ 2. 10.  1. 13.  8. 12. 11.  5.  7.  4.  3.  9.  6.]


In [27]:
import json 
with open("rerun.json", 'w') as fdata:
    json.dump(data,fdata,indent=4)

## Generate LaTex Table in paper

In [74]:
for c,t in quality_tables.items():
    t[1][0] = 'methods \\textbackslash~graphs'
    
    
    values = np.array([r[1:] for r in t[2:]])
    best_indices = values.argmin(axis=0)
    best_values = values.min(axis=0)
    for j in range(len(graphs)):
        best_value = t[best_indices[j]+2][j+1]
        for i in range(len(methods)):
            value = t[i+2][j+1]
            if c == 'crossings':
                if abs(value-best_value) < 1:# \
                    t[i+2][j+1] = f'**{value:d}**'
                    t[i+2][j+1] = f'\\textbf{{{value:d}}}'
                else:
                    t[i+2][j+1] = f'{value:d}'
                    
            else:
                if abs(value-best_value) < 0.001:# \
#                     t[i+2][j+1] = f'**{value:.3f}**'
                    t[i+2][j+1] = f'\\textbf{{{value:.3f}}}'
                else:
                    t[i+2][j+1] = f'{value:.3f}'


In [75]:
from IPython.display import HTML, display
import tabulate

for c,t in quality_tables.items():
#     display(HTML(tabulate.tabulate(t, tablefmt='html')))
#     display(HTML(tabulate.tabulate(t, tablefmt='latex_raw')))
    print(f'%% ==== {c} ====')
    print('\\begin{table}[h]')
    latex_table = tabulate.tabulate(t[1:], tablefmt='latex_raw').replace('lllllllll', 'l|rrrrrrrr')
    latex_table_list = latex_table.split('\n')
    latex_table_list.insert(3, '\\hline')
    latex_table = '\n'.join(latex_table_list)
    print(latex_table)
    print(f'\\caption{{Quality Measures of {utils.criterion_to_title(c)}}}')
    print(f'\\label{{tab:quality-table-{c}}}')
    print('\\end{table}')
    
    print(f'\n\n')


%% ==== stress ====
\begin{table}[h]
\begin{tabular}{l|rrrrrrrr}
\hline
 methods \textbackslash~graphs & dodecahedron   & tree-2-6       & grid-12-24     & spx-teaser     & 494-bus        & grid1          & dwt-307        & dwt-1005       \\
\hline
 neato                         & 0.081          & \textbf{0.078} & \textbf{0.013} & \textbf{0.027} & 0.076          & \textbf{0.062} & 0.083          & \textbf{0.021} \\
 sfdp                          & \textbf{0.080} & 0.133          & 0.024          & 0.052          & 0.099          & 0.071          & \textbf{0.081} & 0.029          \\
 GD2 (ST)                      & \textbf{0.079} & \textbf{0.078} & \textbf{0.013} & \textbf{0.027} & \textbf{0.071} & \textbf{0.062} & 0.082          & \textbf{0.022} \\
 GD2 (NP)                      & 0.153          & 0.122          & 0.062          & 0.184          & 0.154          & 0.134          & 0.159          & 0.167          \\
 GD2 (ST+IL)                   & 0.086          & 0.080          & 0.01